In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/kilm01.xlsx'

In [3]:
data = pd.read_excel(source, skiprows=2)

In [4]:
data.head()

,Country (code),Country,Region,Sub-region (broad),Sub-region (detailed),Income group (code),Income group,Year,Sex (code),Sex,Age group (code),Labour force ('000),Population ('000),Labour force participation rate (%),Repository (code),Repository,Type of statistic (code),Type of statistic
0,AFG,Afghanistan,Asia and the Pacific,Southern Asia,NaN,I,Low income,1990,MF,Male and female,15+,3196.350586,6267.271973,51.000668,LFEP,"LFEP Database, 7th edition (January 2016 of th...",ILO,ILO estimates
1,AFG,Afghanistan,Asia and the Pacific,Southern Asia,NaN,I,Low income,1990,MF,Male and female,15-24,1026.528320,2293.822021,44.751873,LFEP,"LFEP Database, 7th edition (January 2016 of th...",ILO,ILO estimates
2,AFG,Afghanistan,Asia and the Pacific,Southern Asia,NaN,I,Low income,1990,MF,Male and female,15-64,3133.075439,6000.932129,52.209812,LFEP,"LFEP Database, 7th edition (January 2016 of th...",ILO,ILO estimates
3,AFG,Afghanistan,Asia and the Pacific,Southern Asia,NaN,I,Low income,1990,MF,Male and female,25-54,1936.388672,3330.002930,58.149757,LFEP,"LFEP Database, 7th edition (January 2016 of th...",ILO,ILO estimates
4,AFG,Afghanistan,Asia and the Pacific,Southern Asia,NaN,I,Low income,1990,MF,Male and female,25-34,910.755432,1549.147095,58.790768,LFEP,"LFEP Database, 7th edition (January 2016 of th...",ILO,ILO estimates


In [10]:
data.columns

Index(['Country (code)', 'Country', 'Region', 'Sub-region (broad)',
       'Sub-region (detailed)', 'Income group (code)', 'Income group', 'Year',
       'Sex (code)', 'Sex', 'Age group (code)', 'Labour force ('000)',
       'Population ('000)', 'Labour force participation rate (%)',
       'Repository (code)', 'Repository', 'Type of statistic (code)',
       'Type of statistic'],
      dtype='object')

In [9]:
# country

In [15]:
country = data[['Country (code)', 'Country']].drop_duplicates().copy()

In [16]:
country.columns = [
    'country', 'name'
]

In [17]:
country['country'] = country['country'].map(to_concept_id)

In [20]:
country.to_csv('../ddf--entities--country.csv', index=False)

In [21]:
# age group

In [40]:
age = data[['Age group (code)']].drop_duplicates().copy()

In [41]:
age.columns = ['age_group']

In [42]:
age['name'] = 'Age ' + age['age_group']

In [43]:
age['age_group'] = age['age_group'].str.replace('+', '_plus')

In [44]:
age.to_csv('../ddf--entities--age_group.csv', index=False)

In [35]:
# sex

In [36]:
sex = data[['Sex (code)', 'Sex']].drop_duplicates().copy()

In [45]:
sex.columns = ['sex', 'name']

sex['sex'] = sex['sex'].map(to_concept_id)

In [46]:
sex.to_csv('../ddf--entities--sex.csv', index=False)

In [47]:
# concepts

In [6]:
discs = ['Name', 'Year', 'Country', 'Sex', 'Age Group']

conc = ["Labour force ('000)", "Population ('000)", "Labour force participation rate (%)"]

In [49]:
cdf = pd.DataFrame([], columns=['concept', 'name', 'concept_type'])

cdf['name'] = [*discs, *conc]

cdf['concept'] = cdf['name'].map(to_concept_id)

In [53]:
cdf.loc[5:, 'concept_type'] = 'measure'
cdf.loc[0, 'concept_type'] = 'string'
cdf.loc[1, 'concept_type'] = 'time'
cdf.loc[2:4, 'concept_type'] = 'entity_domain'

In [55]:
cdf.to_csv('../ddf--concepts.csv', index=False)

In [56]:
# datapoints

In [15]:
dps = data[['Country (code)', 'Sex (code)', 'Age group (code)', 'Year', *conc]].copy()

In [8]:
dps.head()

,Country (code),Sex (code),Age group (code),Year,Labour force ('000),Population ('000),Labour force participation rate (%)
0,AFG,MF,15+,1990,3196.350586,6267.271973,51.000668
1,AFG,MF,15-24,1990,1026.528320,2293.822021,44.751873
2,AFG,MF,15-64,1990,3133.075439,6000.932129,52.209812
3,AFG,MF,25-54,1990,1936.388672,3330.002930,58.149757
4,AFG,MF,25-34,1990,910.755432,1549.147095,58.790768


In [16]:
dps.columns = ['country', 'sex', 'age_group', 'year', *[to_concept_id(x) for x in conc]]

In [17]:
dps['country'] = dps['country'].map(to_concept_id)
dps['sex'] = dps['sex'].map(to_concept_id)

dps['age_group'] = dps['age_group'].str.replace('+', '_plus').map(to_concept_id)

In [18]:
dps = dps.set_index(['country', 'sex', 'age_group', 'year'])

In [19]:
for k, df in dps.items():
    df_ = df.reset_index()
    path = '../ddf--datapoints--{}--by--country--sex--age_group--year.csv'.format(k)
    df_.to_csv(path, index=False)